In [13]:
import pandas as pd

df = pd.DataFrame({
    'number': [0, 1, 2, 3, 4],
    'cust_id': [128, 1201, 9832, 4392, 7472],
    'type': ['a','b','c','d','e'],
    #'cust_age': [13, 21, 19, 21, 60],
    #'cust_sale': [0, 0, 0.2, 0.15, 0.3],
    #'cust_year_birth': [2008, 2000, 2002, 2000, 1961],
    #'cust_order': [1400, 14142, 900, 1240, 8430]
})
df

,number,cust_id,type
0,0,128,a
1,1,1201,b
2,2,9832,c
3,3,4392,d
4,4,7472,e


In [14]:
limit_1 = 1202
df['cust_id'] = df['cust_id'].apply(lambda x: limit_1 if x > limit_1 else x)
df

,number,cust_id,type
0,0,128,a
1,1,1201,b
2,2,1202,c
3,3,1202,d
4,4,1202,e


In [15]:
from datetime import datetime
x = []

for i in reversed(range(5)):
    x.append((datetime.now().date()-pd.Timedelta(days=i)))
    
#df['dates'] = x
df['dates'] = pd.to_datetime(x)
df

,number,cust_id,type,dates
0,0,128,a,2024-02-08
1,1,1201,b,2024-02-09
2,2,1202,c,2024-02-10
3,3,1202,d,2024-02-11
4,4,1202,e,2024-02-12


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   number   5 non-null      int64         
 1   cust_id  5 non-null      int64         
 2   type     5 non-null      object        
 3   dates    5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 288.0+ bytes


In [22]:
df.loc[df['dates'] > pd.Timestamp('2024-02-10'),'cust_id'] = 1500
df

,number,cust_id,type,dates
0,0,128,a,2024-02-08
1,1,1201,b,2024-02-09
2,2,1202,c,2024-02-10
3,3,1500,d,2024-02-11
4,4,1500,e,2024-02-12
